In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os

# Load data
file_path = "/Users/leazeiberts/Downloads/data/Brain - Amygdala_junctions.tsv"
metadata_file_path = "/Users/leazeiberts/Master/Masterarbeit/age_info.tsv"
df = pd.read_csv(file_path, sep="\t")
metadata = pd.read_csv(metadata_file_path, sep="\t")

# Preprocess data
df[['Chromosome', 'Start', 'End']] = df['Name'].str.extract(r'(chr\w+)_(\d+)_(\d+)')
df[['Start', 'End']] = df[['Start', 'End']].apply(pd.to_numeric, errors='coerce')
df.drop(columns=['Description'], inplace=True)
df.dropna(inplace=True)

metadata.dropna(subset=['SUBJID'], inplace=True)
metadata.rename(columns={'SUBJID': 'Sample'}, inplace=True)

# Helper functions
def shannon_entropy(probs):
    #Calculate Shannon entropy.
    probs = np.array(probs, dtype=np.float64)
    probs = probs[probs > 0]
    return -np.sum(probs * np.log2(probs)) if len(probs) > 0 else 0

def calculate_entropy_by_start(df, sample_columns):
    #Entropy per Chromosome + Start, for each sample with >10 reads total.
    grouped = df.groupby(['Chromosome', 'Start'])
    entropies = []

    for (chromosome, start), group in grouped:
        for sample in sample_columns:
            sample_reads = group[sample]
            total_reads = sample_reads.sum()

            if total_reads > 10:
                probs = sample_reads / total_reads
                entropy = shannon_entropy(probs.values)
                entropies.append((chromosome, start, sample, entropy))

    return pd.DataFrame(entropies, columns=['Chromosome', 'Start', 'Sample', 'Entropy'])

def calculate_entropy_by_end(df, sample_columns):
    #Entropy per Chromosome + End, for each sample with >10 reads total.
    grouped = df.groupby(['Chromosome', 'End'])
    entropies = []

    for (chromosome, end), group in grouped:
        for sample in sample_columns:
            sample_reads = group[sample]
            total_reads = sample_reads.sum()

            if total_reads > 10:
                probs = sample_reads / total_reads
                entropy = shannon_entropy(probs.values)
                entropies.append((chromosome, end, sample, entropy))

    return pd.DataFrame(entropies, columns=['Chromosome', 'End', 'Sample', 'Entropy'])

def process_entropy(entropy_df, metadata):
    #Merge entropy data with metadata and clean columns."""
    entropy_df['Base_ID'] = entropy_df['Sample'].str.extract(r'(GTEX-\w+)')
    merged_df = entropy_df.merge(metadata, left_on='Base_ID', right_on='Sample', how='left')
    return merged_df.rename(columns={'Sample_x': 'Sample'}).drop(columns=['Sample_y'])

def calculate_sample_level_correlation(entropy_df, group_col):
    #Calculate Spearman correlation between AGE and Entropy directly at sample level.
    #Only for (chromosome, position) groups with ≥5 samples.
    
    correlations = []

    for (chromosome, position), group in entropy_df.groupby(['Chromosome', group_col]):
        if (
            len(group) >= 5 and
            group['AGE'].nunique() > 1 and
            group['Entropy'].nunique() > 1
        ):
            correlation, _ = spearmanr(group['AGE'], group['Entropy'])
            correlations.append((chromosome, position, correlation))

    return pd.DataFrame(correlations, columns=['Chromosome', group_col, 'Spearman_Correlation'])

# Main processing
sample_columns = df.columns.difference(['Name', 'Chromosome', 'Start', 'End'])

# Calculate entropy for Start and End positions
df_entropy_start = process_entropy(calculate_entropy_by_start(df, sample_columns), metadata)
df_entropy_end = process_entropy(calculate_entropy_by_end(df, sample_columns), metadata)

# Calculate correlations
correlation_start = calculate_sample_level_correlation(df_entropy_start, 'Start')
correlation_end = calculate_sample_level_correlation(df_entropy_end, 'End')

# Output results
print("Sample-Level Spearman Correlations for Start Positions:")
print(correlation_start.head(10))

print("\nSample-Level Spearman Correlations for End Positions:")
print(correlation_end.head(10))


# Add a column to distinguish the type of position
correlation_start['Position_Type'] = 'Start'
correlation_end['Position_Type'] = 'End'

# Rename position column to generic 'Position' for merging
correlation_start.rename(columns={'Start': 'Position'}, inplace=True)
correlation_end.rename(columns={'End': 'Position'}, inplace=True)

# Merge both into a single DataFrame
correlation_combined = pd.concat([correlation_start, correlation_end], ignore_index=True)

# Output the merged DataFrame
print("\nCombined Correlations (Start + End):")
print(correlation_combined.head(10))


In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns

def plot_correlation_histogram(correlation_df, position):
    
    #Plot a histogram of Spearman correlation coefficients for all positions.
    
    plt.figure(figsize=(12, 6))
    sns.histplot(correlation_df['Spearman_Correlation'], bins='auto', kde=True, stat="density", color="blue", edgecolor="black")

    plt.title(f"Distribution of Spearman Correlations for {position} Positions", fontsize=14)
    plt.xlabel("Spearman Correlation Value", fontsize=12)
    plt.ylabel("Density")

    plt.show()

# Plot histogram for all positions
plot_correlation_histogram(correlation_combined, "All")


In [ ]:
from scipy.stats import kstest, anderson, normaltest

def perform_normality_tests_all_positions(correlation_df):
    
    #Perform normality tests for Spearman correlation values across all positions.
    
    data = correlation_df['Spearman_Correlation'].dropna()  # Remove NaNs
    
    if len(data) < 3:
        print("Not enough data points for normality tests.")
        return
    
    print(f"\nNormality Tests for Spearman Correlation Values (All Positions):")
    
    # Kolmogorov-Smirnov Test
    ks_stat, ks_p = kstest(data, 'norm', args=(data.mean(), data.std(ddof=1)))
    print(f"Kolmogorov-Smirnov Test: Statistic={ks_stat:.4f}, p-value={ks_p:.4f}")
    
    # Anderson-Darling Test
    ad_result = anderson(data, dist='norm')
    print(f"Anderson-Darling Test: Statistic={ad_result.statistic:.4f}")
    print(f"Critical Values: {ad_result.critical_values}")
    print(f"Significance Levels: {ad_result.significance_level}")
    
    # D'Agostino and Pearson's Test
    dag_stat, dag_p = normaltest(data)
    print(f"D'Agostino and Pearson's Test: Statistic={dag_stat:.4f}, p-value={dag_p:.4f}")

# Run the normality tests on the combined correlation data
perform_normality_tests_all_positions(correlation_combined)


In [ ]:
from scipy.stats import kstest, anderson, normaltest


def perform_normality_tests_all_positions(correlation_df):
    
    #Perform normality tests for Spearman correlation values across all positions.
    
    data = correlation_df['Spearman_Correlation'].dropna()  
    
    print(f"\nNormality Tests for Spearman Correlation Values (All Positions):")
    
    # Kolmogorov-Smirnov Test
    ks_stat, ks_p = kstest(data, 'norm', args=(data.mean(), data.std()))
    print(f"Kolmogorov-Smirnov Test: Statistic={ks_stat:.4f}, p-value={ks_p:.4f}")
    
    # Anderson-Darling Test
    ad_result = anderson(data, dist='norm')
    print(f"Anderson-Darling Test: Statistic={ad_result.statistic:.4f}")
    print(f"Critical Values: {ad_result.critical_values}")
    print(f"Significance Levels: {ad_result.significance_level}")
    
    # D'Agostino and Pearson's Test
    dag_stat, dag_p = normaltest(data)
    print(f"D'Agostino and Pearson's Test: Statistic={dag_stat:.4f}, p-value={dag_p:.4f}")

# Run the normality tests on the combined correlation data
perform_normality_tests_all_positions(correlation_combined)







In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def detect_percentile_outliers_with_thresholds(df, column, percentile=0.5):
    
    #Detects outliers based on the upper and lower percentile thresholds.
    #Computes and returns the actual threshold values along with the outliers.
    
    lower_threshold = np.percentile(df[column], percentile)
    upper_threshold = np.percentile(df[column], 100 - percentile)

    outliers = df[(df[column] < lower_threshold) | (df[column] > upper_threshold)]
    
    return outliers, lower_threshold, upper_threshold

def plot_correlation_histogram_with_outliers(correlation_df, column, lower_threshold, upper_threshold):
    
    #Plot a histogram of Spearman correlation coefficients and highlight the outlier thresholds.
    
    plt.figure(figsize=(12, 6))
    
    # Histogram of all correlation values
    sns.histplot(correlation_df[column], bins='auto', kde=True, stat="density", color="blue", edgecolor="black", alpha=0.7)

    #Vertical lines for outlier thresholds
    plt.axvline(lower_threshold, color='red', linestyle='dashed', linewidth=2, label=f"Lower Threshold ({lower_threshold:.4f})")
    plt.axvline(upper_threshold, color='red', linestyle='dashed', linewidth=2, label=f"Upper Threshold ({upper_threshold:.4f})")

    # Titles and labels
    plt.title(f"Distribution of {column} with Outlier Thresholds", fontsize=14)
    plt.xlabel(f"{column} Value", fontsize=12)
    plt.ylabel("Frequency", fontsize=12)
    plt.legend()
    plt.grid(axis="y", linestyle="--", alpha=0.7)

    plt.show()

# Detect outliers for Spearman correlations (All positions)
outliers_spearman, lower_threshold, upper_threshold = detect_percentile_outliers_with_thresholds(
    correlation_combined, 'Spearman_Correlation', percentile=0.5
)

# Print summary
print(f"Lower Threshold (0.5%): {lower_threshold:.4f}")
print(f"Upper Threshold (99,5%): {upper_threshold:.4f}")
print(f"Total Outliers Detected: {outliers_spearman.shape[0]}")

# Plot histogram with outlier thresholds
plot_correlation_histogram_with_outliers(correlation_combined, 'Spearman_Correlation', lower_threshold, upper_threshold)


In [ ]:
import pandas as pd
import pybedtools

# 1. Compute global outliers 
global_outliers, global_lower, global_upper = detect_percentile_outliers_with_thresholds(
    correlation_combined, 'Spearman_Correlation', percentile=0.5
)

# 2. Identify the correct position column dynamically
position_col = None
for col in global_outliers.columns:
    if "pos" in col.lower() or "start" in col.lower():  # Covers "Position", "Start_Position", "Start"
        position_col = col
        break

if position_col is None:
    raise KeyError("No suitable position column (e.g., 'Start' or 'Position') found in the DataFrame.")

# 3. Compute Pseudo-End as Position + 1
global_outliers["Pseudo_End"] = global_outliers[position_col] + 1

# 4. Rename columns for BED format
global_outliers["End"] = global_outliers["Pseudo_End"]
global_outliers.rename(columns={position_col: "Start"}, inplace=True)

# 5. Ensure integer type for genomic positions
global_outliers = global_outliers.dropna(subset=["Start", "End"])
global_outliers["Start"] = global_outliers["Start"].astype(int)
global_outliers["End"] = global_outliers["End"].astype(int)

# 6. Separate into lower 5% and upper 95%
lower_outliers = global_outliers[global_outliers["Spearman_Correlation"] < global_lower]
upper_outliers = global_outliers[global_outliers["Spearman_Correlation"] > global_upper]

# 7. Format for BED (Chromosome, Start, End)
lower_pseudo_outliers = lower_outliers[["Chromosome", "Start", "End"]]
upper_pseudo_outliers = upper_outliers[["Chromosome", "Start", "End"]]

# 8. Convert to BED format
lower_pseudo_bed = pybedtools.BedTool.from_dataframe(lower_pseudo_outliers)
upper_pseudo_bed = pybedtools.BedTool.from_dataframe(upper_pseudo_outliers)

# 9. Load reference gene annotations
reference_genes = pybedtools.BedTool("hg38_refseq_genes.bed")

# 10. Intersect lower and upper outliers with reference genes
intersected_lower = lower_pseudo_bed.intersect(reference_genes, wa=True, wb=True).to_dataframe(
    names=["Chromosome", "Start", "End", "Gene"]
)
intersected_upper = upper_pseudo_bed.intersect(reference_genes, wa=True, wb=True).to_dataframe(
    names=["Chromosome", "Start", "End", "Gene"]
)

# 11. Print results
print("Checking intersection results for Lower 0.5% Spearman Outliers...")
print(intersected_lower.head())

print("Checking intersection results for Upper 99.5% Spearman Outliers...")
print(intersected_upper.head())

# 12. Save results
#intersected_lower.to_csv("intersected_pseudo_outliers_spearman_lower1.csv", index=False, sep="\t")
#intersected_upper.to_csv("intersected_pseudo_outliers_spearman_upper99.csv", index=False, sep="\t")

print("Intersection results saved for Lower 0.5% and Upper 99.5% Spearman Outliers.")


In [ ]:
!pip install gprofiler-official


In [ ]:
# Load background genes from Amygdala_junctions.tsv
background_file = file_path
df_background = pd.read_csv(background_file, sep="\t")

# Extract unique gene names from the "Description" column
background_genes = df_background["Description"].dropna().unique().tolist()

# Remove version numbers from Ensembl IDs
background_genes = [gene.split('.')[0] for gene in background_genes]


print(f" Total background genes: {len(background_genes)}")
print(f" Sample background genes: {background_genes[:10]}")


In [ ]:
import pandas as pd
from gprofiler import GProfiler

def process_enrichment_gprofiler(intersected_df, label, background_genes):
    #Perform gene set enrichment analysis using g:Profiler with custom background.
    
    # Extract unique gene names from intersected genes
    unique_genes = intersected_df["Gene"].dropna().unique().tolist()

    print(f"\n {label} - Total unique genes: {len(unique_genes)}")
    print(f" {label} - Sample genes: {unique_genes[:10]}")

    # Run g:Profiler enrichment with a custom background
    if len(unique_genes) > 0:
        gp = GProfiler(return_dataframe=True)
        enrichment_results = gp.profile(
            organism="hsapiens",
            query=unique_genes,
            background=background_genes,  # Use custom background
            sources=["GO:BP", "GO:MF", "GO:CC", "KEGG", "REAC", "WP"]
        )

        # Save results
        output_file = f"gene_enrichment_results_{label}.csv"
        enrichment_results.to_csv(output_file, index=False)
        print(f" {label} - Enrichment results saved in '{output_file}'")
    else:
        print(f" {label} - No genes found for enrichment.")

# Run enrichment analysis with g:Profiler using custom background
process_enrichment_gprofiler(intersected_lower, "spearman_lower_5", background_genes)
process_enrichment_gprofiler(intersected_upper, "spearman_upper_95", background_genes)

print("\n Enrichment analysis completed with custom background.")



In [ ]:
import pandas as pd
from gprofiler import GProfiler

def process_enrichment_gprofiler(intersected_df, label, background_genes):
    #Perform gene set enrichment analysis using g:Profiler with custom background and filtering.
    
    # Extract unique gene names
    unique_genes = intersected_df["Gene"].dropna().unique().tolist()

    print(f"\n {label} - Total unique genes: {len(unique_genes)}")
    print(f" {label} - Sample genes: {unique_genes[:10]}")

    if len(unique_genes) > 0:
        gp = GProfiler(return_dataframe=True)
        enrichment_results = gp.profile(
            organism="hsapiens",
            query=unique_genes,
            background=background_genes,  # Use custom background
            sources=["GO:BP", "GO:MF", "GO:CC", "KEGG", "REAC", "WP"],
            no_evidences=True,  # Removes very general terms
            user_threshold=0.01  # More stringent p-value threshold
        )

        # **Manual Filtering for Large Gene Sets**
        if not enrichment_results.empty:
            enrichment_results = enrichment_results[
                (enrichment_results["precision"] > 0.15) &  # Keep more specific terms
                (enrichment_results["recall"] < 0.05) &  # Remove broad terms covering too many genes
                (enrichment_results["intersection_size"] < 30)  # Exclude overly large terms
            ]  

            # Save results
            output_file = f"gene_enrichment_results_{label}.csv"
            enrichment_results.to_csv(output_file, index=False)
            print(f" {label} - Filtered enrichment results saved in '{output_file}'")
        else:
            print(f" {label} - No significant enrichment results found.")
    else:
        print(f" {label} - No genes found for enrichment.")

# Run enrichment analysis with g:Profiler using custom background
process_enrichment_gprofiler(intersected_lower, "spearman_lower_5", background_genes)
process_enrichment_gprofiler(intersected_upper, "spearman_upper_95", background_genes)

print("\n Enrichment analysis completed with custom background and filtering.")


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def plot_enrichment_results(file_path, label, top_n=10):
    #Plot top N enriched terms from g:Profiler results.
    
    # Load enrichment results, auto-detecting delimiter
    df = pd.read_csv(file_path, sep=None, engine="python")

    # Print column names to verify correct loading
    print(f"\nColumns in {file_path}: {df.columns.tolist()}")

    # Check if expected columns exist
    expected_columns = ["p_value", "name", "source"]
    missing_columns = [col for col in expected_columns if col not in df.columns]
    
    if missing_columns:
        print(f"❌ Missing expected columns: {missing_columns}")
        print(f"⚠ Check the file format and column names: {file_path}")
        return

    # Check if the file contains any data
    if df.empty:
        print(f" No enriched terms found for {label}.")
        return

    # Sort by p-value and select the top terms
    df = df.sort_values(by="p_value", ascending=True).head(top_n)

    # Plot bar chart
    plt.figure(figsize=(10, 6))
    sns.barplot(
        y=df["name"], 
        x=-np.log10(df["p_value"]), 
        hue=df["source"],
        dodge=False
    )

    plt.xlabel("-log10(p-value)", fontsize=12)
    plt.ylabel("Enriched Terms", fontsize=12)
    plt.title(f"Top {top_n} Enriched Terms - {label}", fontsize=14)
    plt.legend(title="Source")
    plt.gca().invert_yaxis()  # Flip the order to show the most significant term on top
    plt.show()

# Plot results for both lower and upper Spearman outliers
plot_enrichment_results("gene_enrichment_results_spearman_lower_5.csv", "Lower 0.5% Outliers")
plot_enrichment_results("gene_enrichment_results_spearman_upper_95.csv", "Upper 99.5% Outliers")
